In [1]:
pip install python-mnist==0.5  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import torch.nn as nn

import torch.optim as optim
import torch.utils.data as data_utils

In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
conv2 = nn.Conv2d(32, 64, 3, 1, padding=1)

In [4]:
print(conv1)
print(conv2)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


- stride에 아무것도 안 주는 경우, default에 1이 들어가는 것을 알 수 있음

# CNN

Input -> (CNN, Batch Normalization, ReLU, Maxpooling) *n -> Fully Connected -> Output

In [5]:
input_data = torch.Tensor(1, 1, 28, 28)
print(input_data.shape)

torch.Size([1, 1, 28, 28])


$$shape = \displaystyle \frac{(Input shape) - (kernel size)+ 2*(padding)}{stride} + 1$$

input shape = (28, 28)이고 <br>
첫번째 Convolution Layer를 통과하고 (28,28) - (3,3) + (2,2) + (1,1) = (28,28) <br>
즉, channel 갯수만 늘고, shape는 변하지 않음

In [6]:
print(conv1(input_data).shape)

torch.Size([1, 32, 28, 28])


[**nn.MaxPool2d(2)**]

nn.MaxPool2d(2)의 경우, kernel size가 2라면 stride도 2로 고정.<br>
padding은 물론 0 <br>

즉, ((28,28) - (2,2)) / 2 + (1,1)은 (14,14)

In [7]:
pool = nn.MaxPool2d(2)
print(pool(conv1(input_data)).shape)

torch.Size([1, 32, 14, 14])


In [8]:
final_output = conv2(pool(conv1(input_data)))
print(final_output.shape)

torch.Size([1, 64, 14, 14])


In [9]:
out = final_output.view(final_output.shape[0],-1) # view == reshape # view와 reshape 모두 input을 원하는 차원의 형태로 변환
print(out.shape)

torch.Size([1, 12544])


In [10]:
fc = nn.Linear(12544, 10)
fc(out).shape

torch.Size([1, 10])

# MINIST with CNN

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(777)

if device == "cuda":
  torch.cuda.manual_seed_all(777)

print(device)

cpu


In [12]:
from mnist import MNIST
import numpy as np

mnist = MNIST("/content/drive/MyDrive/Colab Notebooks/data/raw")

x_train, y_train = mnist.load_training()
x_test, y_test = mnist.load_testing()

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

print("x_train 의 shape={}, y_train 의 shape={}".format(x_train.shape,y_train.shape))
print("x_test 의 shape={}, y_test 의 shape={}".format(x_test.shape,y_test.shape))

x_train 의 shape=(60000, 784), y_train 의 shape=(60000,)
x_test 의 shape=(10000, 784), y_test 의 shape=(10000,)


# Conver array data into Tensor Dataset

In [13]:
train_data = data_utils.TensorDataset(torch.FloatTensor(x_train), torch.FloatTensor(y_train))

batch_size = 1000

trainloader = data_utils.DataLoader(train_data, batch_size = batch_size, shuffle=True)

# CNN

  (layer1): Sequential( <br>
    (0): Conv Layer (kernel size = 1, stride = 1, padding = 1) <br>
    (1): Batch Normalization<br>
    (2): ReLU <br>
    (3): MaxPooling (kernel size = 2) <br>
<br>
  (layer2): Sequential( <br>
    (0): Conv Layer (kernel size = 5, stride = 2, padding = 0) <br>
    (1): Batch Normalization<br>
    (2): ReLU <br>
    (3): MaxPooling (kernel size = 2) <br>
<br>
  (fc): Linear(in_features=???, out_features=10) <br>

In [16]:
class CNN(nn.Module):
    def __init__(self):
       super(CNN, self).__init__()

       # -> CONN/FC -> BatchNorm -> ReLU(or other activation) -> Dropout -> Conv/FC -> output

       self.layer1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=1, stride=1, padding=1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(),
                                   nn.MaxPooling(2))
       
       self.layer2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=0),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
       
       self.fc = nn.Linear(????, 10) # ????? 값 채워보기

    def forward(self, x):
      cnn1 = self.layer1(x)
      cnn2 = self.layer2(cnn1)

      output = cnn2.view(cnn2.shape[0], -1)
      output = self.fc(output)

      return output

## Convolutional Layer의 Output을 쉽게 구하는 방법

- dummy data 넣어보기
- dummy data를 input image와 같은 size로 만들고 output size를 알아보면 됨.
- mnist 이미지 데이터가 28x28이므로 사이즈는 물론 28x28이어야 함

In [18]:
dummy_data = torch.Tensor(1000, 1, 28, 28).to(device) # batch_size는 임의로 지정 # 1000으로 지정
print(dummy_data.shape)

torch.Size([1000, 1, 28, 28])


- 중간에 conv layer를 지나고 난 후, shape를 확인할 수 있도록 conv layer까지만 model 짜보기

In [25]:
class dummy_CNN(nn.Module):
    def __init__(self):
       super(dummy_CNN, self).__init__()

       self.layer1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=1, stride=1, padding=1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
       
       self.layer2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=0),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
       
    def forward(self,x):
       conv1 = self.layer1(x)
       conv2 = self.layer2(conv1)

       return conv2

In [26]:
dummy_model = dummy_CNN().to(device)

In [27]:
dummy_model(dummy_data).shape

torch.Size([1000, 128, 3, 3])

- Conv Layer의 Output은 1000(batch size) x 128(output channel) x 3 x 3을 나타냄

In [30]:
class CNN(nn.Module):
    def __init__(self):
       super(CNN, self).__init__()

       # -> CONN/FC -> BatchNorm -> ReLU(or other activation) -> Dropout -> Conv/FC -> output

       self.layer1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=1, stride=1, padding=1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
       
       self.layer2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=0),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2))
       
       self.fc = nn.Linear(3*3*128, 10) 

    def forward(self, x):
      cnn1 = self.layer1(x)
      cnn2 = self.layer2(cnn1)

      output = cnn2.view(cnn2.shape[0], -1) # batch_size는 유지하고 나머지는 다 펼친다는 의미.
      output = self.fc(output)

      return output

In [31]:
model = CNN().to(device)

# Parameters

In [32]:
epoch = 15
learning_rate = 0.001
weight_decay = 1e-5

In [34]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train

In [ ]:
total_batch = len(trainloader) # 전체 mini batch의 갯수

for num_epoch in range(epoch):
  avg_loss = 0

  for batch_num, (images, labels) in enumerate(trainloader):
    x = images.to(device)
    y = labels.to(device)

    x = x.reshape(1000,1,28,28)
    
    optimizer.zero_grad()

    predict = model(x)

    loss = loss_function(predict, y.long())

    loss.backward()
    optimizer.step()

    avg_loss = avg_loss + (loss/total_batch)

  print("Epoch = {} loss = {:.4f}".format(num_epoch+1, avg_loss.item()))

Epoch = 1 loss = 0.5364
Epoch = 2 loss = 0.1750
Epoch = 3 loss = 0.1262
Epoch = 4 loss = 0.1022
Epoch = 5 loss = 0.0864
Epoch = 6 loss = 0.0764
Epoch = 7 loss = 0.0681
Epoch = 8 loss = 0.0624
Epoch = 9 loss = 0.0570
Epoch = 10 loss = 0.0536
Epoch = 11 loss = 0.0494
Epoch = 12 loss = 0.0458
Epoch = 13 loss = 0.0429
Epoch = 14 loss = 0.0405
Epoch = 15 loss = 0.0375


In [ ]:
torch.save(model.state_dict(), './pre_trained/CNN_MNIST.pth')

FileNotFoundError: ignored

In [ ]:
model = CNN().to(device)

In [ ]:
model.load_state_dict(torch.load('./pre_trained/CNN_MNIST.pth', map_location=device))

# Test

In [ ]:
test_data = data_utils.TensorDataset(torch.FloatTensor(x_test), torch.FloatTensor(y_test))

batch_size = 1000
testloader = data_utils.DataLoader(train_data, batch_size = batch_size, shuffle=False)

In [ ]:
with torch.no_grad():

  num_total_data = 0
  correct = 0

  for batch_idx, (images, labels) in enumerate(testloader):

    images = images.to(device)
    labels = labels.to(device)

    images = images.reshape(batch_size, 1, 28, 28)

    outputs = model(images).to(device)

    outputs_softmax = torch.nn.functional.softmax(outputs, dim=1)

    predicted = torch.argmax(outputs_softmax, dim=1)

    num_total_data = num_total_data + len(images)

    answer = sum(labels==predicted).item()
    correct = correct + answer

print("CNN을 이용한 모델의 정확도는 {:.5f}".format((correct/num_total_data)*100))